In [1]:
# | default_exp _components.aiokafka_producer_manager

In [12]:
# | export

import asyncio
from contextlib import asynccontextmanager, contextmanager
from typing import *

import anyio
from aiokafka import AIOKafkaProducer
from anyio.streams.memory import MemoryObjectReceiveStream, MemoryObjectSendStream

from fastkafka._components.logger import get_logger

In [6]:
import unittest.mock
from os import environ

import nest_asyncio

from fastkafka._components.logger import supress_timestamps
from fastkafka.testing import (
    create_and_fill_testing_topic,
    nb_safe_seed,
    true_after,
)

In [7]:
seed = nb_safe_seed("_components.aiokafka_producer_loop")

In [8]:
# | notest

# allows async calls in notebooks
nest_asyncio.apply()

In [9]:
# | export

logger = get_logger(__name__)

In [10]:
supress_timestamps()
logger = get_logger(__name__, level=1)
logger.info("ok")

[INFO] __main__: ok


In [11]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {"bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}"}

In [19]:
# | export


@asynccontextmanager
async def _aiokafka_producer_manager(  # type: ignore # Argument 1 to "_aiokafka_producer_manager" becomes "Any" due to an unfollowed import  [no-any-unimported]
    producer: AIOKafkaProducer,
    *,
    max_buffer_size: int = 10_000,
) -> AsyncGenerator[MemoryObjectSendStream[Any], None]:
    """Write docs

    Todo: add batch size if needed

    """

    logger.info("_aiokafka_producer_manager(): Starting...")

    async def send_message(receive_stream: MemoryObjectReceiveStream) -> None:
        async with receive_stream:
            async for topic, msg in receive_stream:
                fut = await producer.send(topic, msg)
                msg = await fut

    send_stream, receive_stream = anyio.create_memory_object_stream(
        max_buffer_size=max_buffer_size
    )

    logger.info("_aiokafka_producer_manager(): Starting send_stream")
    asyncio.create_task(send_message(receive_stream))
    async with send_stream:
        yield send_stream
        logger.info("_aiokafka_producer_manager(): Exiting send_stream")

    logger.info("_aiokafka_producer_manager(): Finished.")

In [20]:
@contextmanager
def mock_AIOKafkaProducer_send():
    with unittest.mock.patch("__main__.AIOKafkaProducer.send") as mock:

        async def _f():
            pass

        mock.return_value = asyncio.create_task(_f())

        yield mock

In [21]:
num_msgs = 15
topic = "topic"
msg = b"msg"
msgs = [(topic, msg) for _ in range(num_msgs)]
calls = [unittest.mock.call(topic, msg) for _ in range(num_msgs)]

with mock_AIOKafkaProducer_send() as send_mock:
    producer = AIOKafkaProducer()
    async with _aiokafka_producer_manager(producer) as send_stream:
        for msg in msgs:
            send_stream.send_nowait(msg)

        await asyncio.sleep(10)

        await producer.stop()

    send_mock.assert_has_calls(calls)

[INFO] __main__: _aiokafka_producer_manager(): Starting...
[INFO] __main__: _aiokafka_producer_manager(): Starting send_stream
[DEBUG] aiokafka.producer.producer: The Kafka producer has closed.
[INFO] __main__: _aiokafka_producer_manager(): Exiting send_stream
[INFO] __main__: _aiokafka_producer_manager(): Finished.


In [22]:
# | export


class AIOKafkaProducerManager:
    def __init__(self, producer: AIOKafkaProducer, *, max_buffer_size: int = 1_000):  # type: ignore
        self.producer = producer
        self.max_buffer_size = max_buffer_size

    async def start(self) -> None:
        logger.info("AIOKafkaProducerManager.start(): Entering...")
        await self.producer.start()
        self.producer_manager_generator = _aiokafka_producer_manager(self.producer)
        self.send_stream = await self.producer_manager_generator.__aenter__()
        logger.info("AIOKafkaProducerManager.start(): Finished.")

    async def stop(self) -> None:
        # todo: try to flush messages before you exit
        logger.info("AIOKafkaProducerManager.stop(): Entering...")
        await self.producer_manager_generator.__aexit__(None, None, None)
        logger.info("AIOKafkaProducerManager.stop(): Stoping producer...")
        await self.producer.stop()
        logger.info("AIOKafkaProducerManager.stop(): Finished")

    def send(self, topic: str, msg: bytes) -> None:
        self.send_stream.send_nowait((topic, msg))

In [23]:
producer = AIOKafkaProducer(bootstrap_servers=kafka_config["bootstrap.servers"])
manager = AIOKafkaProducerManager(producer)
await manager.start()
manager.send("topic", b"msg")
await manager.stop()
logger.info("Stopped")

[INFO] __main__: AIOKafkaProducerManager.start(): Entering...
[DEBUG] aiokafka.producer.producer: Starting the Kafka producer
[DEBUG] aiokafka: Attempting to bootstrap via node at davor-fastkafka-kafka-1:9092
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=davor-fastkafka-kafka-1 port=9092> Request 1: MetadataRequest_v0(topics=[])
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=davor-fastkafka-kafka-1 port=9092> Response 1: MetadataResponse_v0(brokers=[(node_id=1001, host='ff698b6fbc7f', port=9092), (node_id=1003, host='ae0c8f0371ac', port=9092), (node_id=1002, host='ed3c61e60058', port=9092)], topics=[(error_code=0, topic='prediction_request', partitions=[(error_code=0, partition=0, leader=1003, replicas=[1003, 1001, 1002], isr=[1003, 1001, 1002]), (error_code=0, partition=5, leader=1002, replicas=[1002, 1001, 1003], isr=[1002, 1001, 1003]), (error_code=0, partition=10, leader=1001, replicas=[1001, 1003, 1002], isr=[1001, 1003, 1002]), (error_code=0, partition=20, leader=1002, replica

[DEBUG] aiokafka.cluster: Updated cluster metadata to ClusterMetadata(brokers: 3, topics: 4, groups: 0)
[DEBUG] aiokafka.conn: Closing connection at davor-fastkafka-kafka-1:9092
[DEBUG] aiokafka: Received cluster metadata: ClusterMetadata(brokers: 3, topics: 4, groups: 0)
[DEBUG] aiokafka: Initiating connection to node 1001 at ff698b6fbc7f:9092
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=ff698b6fbc7f port=9092> Request 1: ApiVersionRequest_v0()
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=ff698b6fbc7f port=9092> Response 1: ApiVersionResponse_v0(error_code=0, api_versions=[(api_key=0, min_version=0, max_version=9), (api_key=1, min_version=0, max_version=12), (api_key=2, min_version=0, max_version=6), (api_key=3, min_version=0, max_version=11), (api_key=4, min_version=0, max_version=5), (api_key=5, min_version=0, max_version=3), (api_key=6, min_version=0, max_version=7), (api_key=7, min_version=0, max_version=3), (api_key=8, min_version=0, max_version=8), (api_key=9, min_version=

[DEBUG] aiokafka.conn: Closing connection at ff698b6fbc7f:9092
[DEBUG] aiokafka.producer.producer: Kafka producer started
[INFO] __main__: _aiokafka_producer_manager(): Starting...
[INFO] __main__: _aiokafka_producer_manager(): Starting send_stream
[INFO] __main__: AIOKafkaProducerManager.start(): Finished.
[INFO] __main__: AIOKafkaProducerManager.stop(): Entering...
[INFO] __main__: _aiokafka_producer_manager(): Exiting send_stream
[INFO] __main__: _aiokafka_producer_manager(): Finished.
[INFO] __main__: AIOKafkaProducerManager.stop(): Stoping producer...
[DEBUG] aiokafka: Initiating connection to node 1002 at ed3c61e60058:9092
[DEBUG] aiokafka.conn: Closing connection at ff698b6fbc7f:9092
[DEBUG] aiokafka.producer.producer: The Kafka producer has closed.
[INFO] __main__: AIOKafkaProducerManager.stop(): Finished
[INFO] __main__: Stopped
